In [1]:
import pandas as pd
import numpy as np

In [2]:
E = 2.1e11
mu = 0.3
B = 5.0
rho = 7850
rho_2 = 20150
stiff= [1.2e10, 1.2e10, 1.2e10, 1.2e10, 1.5e7, 1.5e7, 8.0e8]
mesh_size = 1.0

dict_section = {'upper chord':   {'area':0.056, 'i22':0.0031,   'i11':0.00067,  'part' : 'Part-1'},
                'lower chord':   {'area':0.02,  'i22':0.0063,   'i11':0.00021,  'part' : 'Part-1'},
                'stringer':      {'area':0.02,  'i22':0.000627, 'i11':0.000207, 'part' : 'Part-1'},
                'transverse':    {'area':0.026, 'i11':0.00361,  'i22':0.000203, 'part' : 'Part-1'},
                'strut':         {'area':0.02,  'i11':0.0028,   'i22':0.000625, 'part' : 'Part-1'},
                'lower bracing': {'area':0.0036,'i11':1.09e-05, 'i22':8e-06,    'part' : 'Part-1'},
                'upper bracing': {'area':0.0049,'i11':4.38e-06, 'i22':2.38e-06, 'part' : 'Part-1'},                
                'vertical':      {'area':0.01,  'i22':0.000115, 'i11':5.49e-05, 'part' : 'Part-2'},
                'diagonal':      {'area':0.014, 'i22':0.000278, 'i11':0.000124, 'part' : 'Part-2'}
                }

In [3]:
row={}
row['E'] = E
row['mu'] = mu
row['rho'] = rho
row['rho_2'] = rho_2
row['B'] = B
row['n_part_damage'] = 0
row['i_part_damage'] = 0
row['damage_ratio'] = 0
row['damage_ratio_fluctuation']=0

for i in range(1,8):
    row['k'+str(i)] = stiff[i-1]

row['mesh_size'] = mesh_size
row['corrosion_factor'] = 0
row['section_corrosion']=0
row['temperature_factor']=0
row['weight_factor'] = 0

for key, value in dict_section.items():
    for k in value:
        row[key + '_' + k] = value[k]

row['mesh_size'] = 1.0      
row['jobname'] = 1.0


In [6]:
list_weight_factor = [0, -0.05, 0.05]   
list_temperature_factor = [0, -0.02, 0.02]
list_corrosion_factor = [0.0, -0.05, 0.05]
list_section_corrosion = ['All', 'upper chord', 'lower chord', 'stringer', 'transverse', 'strut', 'lower bracing', 'upper bracing',                
                'vertical', 'diagonal']
list_damage_ratio = [0.0, 0.05, 0.15, 0.25]
list_damage_ratio_fluctuation= [0, 0.01]
list_damage_part = [1, 2, 3, 4, 5, 6, 7]




In [16]:
def damage_assign(row, j, df):
    for d in list_damage_ratio:
        row['damage_ratio'] = d
        
        for p in list_damage_part:
            row['k'+str(p)] = stiff[i-1] 
        
        if d==0.0: 
            row['n_part_damage'] = 0
            row['i_part_damage'] = 0            
            row['damage_ratio_fluctuation']=0
            j += 1
            df.loc[j] = row  
            if j%1000 ==0:  print(j)
        else:
            for p in list_damage_part:
                for drf in list_damage_ratio_fluctuation:
                    dam = d + drf                     
                    row['k'+str(p)] = dam* stiff[i-1]                                        
                    row['n_part_damage'] = 1
                    row['i_part_damage'] = p
                    row['damage_ratio_fluctuation']=drf
                    j += 1
                    df.loc[j] = row
                    if j%1000 ==0:  print(j)
    return j, df

j = 0
df = pd.DataFrame(columns=list(row))

for w in list_weight_factor:
    row['weight_factor'] = w
    row['rho_2'] = (1.0 - w) * rho_2
    for t in list_temperature_factor:
        row['temperature_factor']= t
        row['E'] = (1.0 - t) * E 
        for c in list_corrosion_factor:
            row['corrosion_factor'] = c
            if c == '0.0':
                for key, value in dict_section.items():
                    for k in value:
                        if k != 'part':
                            row[key + '_' + k] = value[k] 
                            
                j, df = damage_assign(row, j, df)
                
            else:                
                for isect in list_section_corrosion:
                    row['section_corrosion']= isect
                    if isect == 'All':
                        for key, value in dict_section.items():
                            for k in value:
                                if k != 'part':
                                    row[key + '_' + k] = (1.0 -c) * value[k] 
                    else:
                        for k in dict_section[isect]:
                            if k != 'part':
                                row[isect + '_' + k] = (1.0 -c) * value[k] 
                                
                    j, df = damage_assign(row, j, df)
                                    
                                

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [17]:
df.to_csv('database_NamO.csv')

In [9]:
df.head()

,E,mu,rho,rho_2,B,n_part_damage,i_part_damage,damage_ratio,damage_ratio_fluctuation,k1,...,upper bracing_part,vertical_area,vertical_i22,vertical_i11,vertical_part,diagonal_area,diagonal_i22,diagonal_i11,diagonal_part,jobname
1,2.100000e+11,0.3,7850,20150.0,5.0,0,0,0,0,800000000.0,...,Part-1,0.01,0.000115,0.000055,Part-2,0.014,0.000278,0.000124,Part-2,1.0
2,2.100000e+11,0.3,7850,20150.0,5.0,0,0,0,0,40000000.0,...,Part-1,0.01,0.000115,0.000055,Part-2,0.014,0.000278,0.000124,Part-2,1.0
3,2.100000e+11,0.3,7850,20150.0,5.0,0,0,0,0,48000000.0,...,Part-1,0.01,0.000115,0.000055,Part-2,0.014,0.000278,0.000124,Part-2,1.0
4,2.100000e+11,0.3,7850,20150.0,5.0,0,0,0,0,48000000.0,...,Part-1,0.01,0.000115,0.000055,Part-2,0.014,0.000278,0.000124,Part-2,1.0
5,2.100000e+11,0.3,7850,20150.0,5.0,0,0,0,0,48000000.0,...,Part-1,0.01,0.000115,0.000055,Part-2,0.014,0.000278,0.000124,Part-2,1.0


In [28]:
df.loc[2000]

E                               2.142e+11
mu                                    0.3
rho                                  7850
rho_2                               20150
B                                       5
n_part_damage                           1
i_part_damage                           4
damage_ratio                         0.15
damage_ratio_fluctuation                0
k1                               1.28e+08
k2                               1.28e+08
k3                               1.28e+08
k4                                1.2e+08
k5                                  8e+08
k6                                  8e+08
k7                                  8e+08
mesh_size                               1
corrosion_factor                    -0.05
section_corrosion           lower bracing
temperature_factor                  -0.02
weight_factor                           0
upper chord_area                   0.0147
upper chord_i22                 0.0002919
upper chord_i11                 0.

In [ ]:
c = 0
if c != 0: 
    print('a')

In [ ]:
E1 = E
mu1 = mu
rho1 = rho
mu2 = mu
rho2 = rho
ratio_reduction = 0
E_reduction = E1 * (1 - ratio_reduction)
L = 2.5 
N_part = 10
size_mesh = 0.05
BC_left = 'pinned'
BC_right = 'pinned'
section_type = 'I-section'
b = 0.055
h = 0.1
t = 0.003
jobname = 'jobbeam_0'
n_part_damage = 0
i_part_damage = -1
if n_part_damage == 0:
    E2 = E1
    ratio_reduction = -1

label = 'healthy'
if ratio_reduction > 0.05:    
    label = 'unhealthy'


In [ ]:
data = [L, N_part, size_mesh, E, mu, rho, E_reduction, BC_left, BC_right, section_type, b, h, t, jobname, n_part_damage, i_part_damage, ratio_reduction, label]

In [ ]:
len(data), len(columns)

In [ ]:
df.loc[0] = data

In [ ]:
df

In [ ]:
list_ratio = np.array([0, 0.05, 0.15, 0.25])
list_part = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
list_E = [2.1e10, 2.1e11, 2.1e12]
list_rho = [1000, 7850, 15000]
list_h = [0.05, 0.1, 0.2]
list_t = [0.001, 0.003, 0.005]
list_BC_left = ['pinned', 'fixed']
list_BC_right = ['pinned', 'fixed']


In [ ]:

i_df = 0

for E in list_E:
    for rho in list_rho:
        for h in list_h:
            for t in list_t:                           
                for BC_left in list_BC_left:
                    for BC_right in list_BC_right:                        
                        for ratio_reduction in list_ratio:
                            E_reduction = E * (1.0 - ratio_reduction)
                            label = 'healthy'
                            if ratio_reduction > 0.05:    
                                label = 'unhealthy'
                        
                            if ratio_reduction == 0:
                                n_part_damage = 0
                                i_part_damage = -1
                                
                                jobname = 'jobbeam_' + str(i_df)
                                data = [L, N_part, size_mesh, E, mu, rho, E_reduction, BC_left, BC_right, section_type, b, h, t, jobname, n_part_damage, i_part_damage, ratio_reduction, label]    
                                df.loc[i_df] = data
                                i_df += 1                                                               
                            else:
                                for i_part_damage in list_part:
                                    n_part_damage = 1
                                    
                                    jobname = 'jobbeam_' + str(i_df) 
                                    data = [L, N_part, size_mesh, E, mu, rho, E_reduction, BC_left, BC_right, section_type, b, h, t, jobname, n_part_damage, i_part_damage, ratio_reduction, label]    
                                    df.loc[i_df] = data        
                                    i_df += 1
                                    
                                    
    

    


In [ ]:
df.head()

In [ ]:
df.to_csv('jobbeam_list.csv')

In [ ]:
data = df.loc[1]

In [ ]:
data.E

In [ ]:
dff = pd.read_csv('jobbeam_list.csv')

In [ ]:
dff.head()

In [ ]:
dff.tail()

In [ ]:
dff.loc[1]['L']